In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

/tmp/ipykernel_5360/3321561094.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
flag_project = 1 # 0--regular ; 1--repartrition

In [3]:
if flag_project == 0:
    file_project = 'a-project.yml'
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v3.0/output_repar_v9.1_02-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
dir_disp_autopick = dir_project + info_basic['dir_disp']

In [7]:
key_subworks = info_basic['key_subworks']
key_subworks = info_basic['key_subworks_repick']
key_subworks

['554--23-16',
 '560--29-16',
 '610--39-17',
 '409--43-12',
 '60--21-04',
 '644--39-18',
 '496--42-14',
 '600--26-17',
 '615--45-17',
 '465--07-14',
 '524--30-15',
 '109--40-05',
 '435--24-13',
 '64--25-04',
 '612--41-17',
 '571--41-16',
 '516--19-15',
 '632--18-18',
 '621--51-17',
 '594--18-17',
 '189--39-07',
 '425--13-13',
 '412--47-12',
 '523--29-15',
 '483--29-14',
 '433--22-13',
 '620--50-17',
 '552--21-16',
 '155--44-06',
 '643--38-18',
 '651--46-18',
 '110--41-05',
 '231--39-08',
 '636--24-18',
 '437--26-13',
 '113--47-05',
 '631--17-18',
 '618--48-17',
 '421--08-13',
 '597--23-17',
 '646--41-18',
 '601--27-17',
 '65--26-04',
 '428--16-13',
 '415--50-12',
 '93--19-05',
 '116--51-05',
 '563--33-16',
 '408--42-12',
 '542--51-15',
 '490--36-14',
 '370--50-11',
 '78--43-04',
 '489--35-14',
 '567--37-16',
 '321--41-10',
 '115--49-05',
 '426--14-13',
 '578--48-16',
 '462--51-13',
 '57--18-04',
 '653--49-18',
 '429--17-13',
 '519--24-15',
 '609--38-17',
 '617--47-17',
 '629--15-18',
 

### create workspace for Dispernet

In [8]:
info_basic['dir_inv_dispernet'] = 'inv_dispernet/'
dir_inv = dir_project + info_basic['dir_inv_dispernet']
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [9]:
dir_disp = dir_inv + 'data/'
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)

### Autopick

In [10]:
flag_f = [5,15]

In [11]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    mode = np.zeros(len(ff_this))
    np.savetxt(dir_disp+'ds_'+key_subwork+'curve.txt',np.c_[ff_this,cc_this,mode], fmt='%.6f  %.6f  %i')

In [12]:
for key_subwork in key_subworks:
    #print(key_subwork)
    transfer_autopick(key_subwork)

554--23-16
560--29-16
610--39-17
409--43-12
60--21-04


644--39-18
496--42-14
600--26-17
615--45-17
465--07-14
524--30-15
109--40-05
435--24-13
64--25-04
612--41-17
571--41-16
516--19-15
632--18-18
621--51-17
594--18-17
189--39-07
425--13-13
412--47-12
523--29-15
483--29-14
433--22-13
620--50-17
552--21-16
155--44-06
643--38-18
651--46-18
110--41-05
231--39-08
636--24-18
437--26-13
113--47-05
631--17-18
618--48-17
421--08-13
597--23-17
646--41-18
601--27-17
65--26-04
428--16-13
415--50-12
93--19-05
116--51-05
563--33-16
408--42-12
542--51-15
490--36-14
370--50-11
78--43-04
489--35-14
567--37-16
321--41-10
115--49-05
426--14-13
578--48-16
462--51-13
57--18-04
653--49-18
429--17-13
519--24-15
609--38-17
617--47-17
629--15-18
434--23-13
108--39-05
75--40-04
580--50-16
641--36-18
117--52-05
630--16-18
572--42-16
81--47-04
165--13-07
638--27-18
595--20-17
84--50-04
654--50-18
576--46-16
657--54-18
561--31-16
551--19-16
525--31-15
427--15-13
482--28-14
453--42-13
80--46-04
581--51-16
411--46-12
607--36-17
